In [13]:
import os , os.path #osを読み込む
import sys
import pandas as pd #xlsxのファイルをcsvにする
import ast
import requests
from bs4 import BeautifulSoup
import json
from urllib import request
import csv
import glob

In [14]:
#ここから係り受け構文解析
APPID = "dj00aiZpPUVzanI5WmtzbjExVyZzPWNvbnN1bWVyc2VjcmV0Jng9NTQ-"  # <-- ここにあなたのアプリケーションIDを設定してください。
URL = "https://jlp.yahooapis.jp/DAService/V2/parse"

#post(???)で係り受け解析をした結果を得ることができる
def post(query):
    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Yahoo AppID: {}".format(APPID),
    }
    param_dic = {
        "id": "1234-1",
        "jsonrpc": "2.0",
        "method": "jlp.daservice.parse",
        "params": {"q": query},
    }
    params = json.dumps(param_dic).encode()
    req = request.Request(URL, params, headers)
    with request.urlopen(req) as res:
        body = res.read()
    return body.decode()

In [15]:
#シリーズごとに製品を絞り込んでいる
df3 = pd.read_excel("カメラ辞書.xlsx", sheet_name=0)
df3.to_csv("カメラ辞書.csv", index=False, header=None, encoding='utf_8_sig')
with open("カメラ辞書.csv", encoding='utf_8_sig') as fp:
    camera_query = df3.query('シリーズ == "EOS" & 種別 == "ミラーレス一眼"')
    siborikomi = camera_query["品名"]#絞り込んだ結果の品名を取り出す”EOS R5_body”とか

In [16]:
selected_products = []#絞り込んだ結果をリストにする
for selects in siborikomi:
    selected_products.append(selects)
print(selected_products)

['EOS R5_body']


In [208]:
for series in selected_products:#品名ごとに抜き出す
    with open("./review/{}.csv".format(series), "w"):#csvファイルを作成
        pass#処理はしない

    with open("./review/{}.csv".format(series), "a", newline="")as fp: #newline=""はcsvファイルの一行詰めるため "a"は追記可能
        writer = csv.writer(fp)

        files = glob.glob("./review/*/*/{}_*_*.txt".format(series)) #*に入るのは何でもいい
        for file in files:
            f = open(file,"r")
            review = f.read()
            response = post(review)#post(???)で係り受け解析をした結果を得ることができる
            response = ast.literal_eval(post(review))#文字列を辞書型に変換
            f.close()
    #        with open("./review/{}_word.csv".format(series), "a", newline="")as fp: #newline=""はcsvファイルの一行詰めるため "a"は追記可能
    #            writer = csv.writer(fp)
    #            for element in response["result"]["chunks"]:
    #                tokens_item = element["tokens"]#品詞や原形を抜き出す（値のみ）
    #                writer.writerows(tokens_item)


    
#id_listとhead_listはいらないかも
            id_list = []
            head_list = []
            toword_list = [] ##係り先の言葉が入るリスト
            
            for element in response["result"]["chunks"]:
                #係り受け解析の結果から各要素を抜き出す
                id_item = element["id"]#係り先を抜き出す（値のみ）
                id_list.append(id_item)
                
                head_item = element["head"]#係り先を抜き出す（値のみ）
                head_list.append(head_item)
                
                tokens_item = element["tokens"]#品詞や原形を抜き出す（値のみ）
                
                ##係り先
                toword_item = response["result"]["chunks"][head_item]["tokens"]#head_itemの数字のtokensを抜き出す
                tokihon = []
                for tokihonkei in toword_item:
                    if tokihonkei[3] == "特殊":#品詞が特殊なら省く
                        continue
                    tokihon.append(tokihonkei[0])#言葉をリストに追加
                totokens = ''.join(tokihon)#リストの中身を結合
                #print(tokens_item)        
                ##

                #tokensの処理
                kihon = []
                for kihonkei in tokens_item:
                    if kihonkei[3] == "特殊":#品詞が特殊なら省く
                        continue
                    kihon.append(kihonkei[0])#言葉をリストに追加
                tokens = ''.join(kihon)#リストの中身を結合
                #print(tokens_item)        
                
                tokens_totokens = tokens + totokens
                #print(tokens_totokens)
                
                
                word_list = [id_item, head_item, tokens, totokens, tokens_totokens]#配列型にする
                all_list = [word_list]#二次元配列にする
                #print(all_list)
                writer.writerows(all_list)

                
#            for number,kakarisaki in enumerate(head_list):
#                print(number, kakarisaki)
            
#            tokens_list = []
#            for id_number in headlist:
                
             #係り受け先の言葉と合体させる
#            if id_item == head_item:
#                print(tokens_item)
        
            
            
            
            
            
                    #print(element["head"])
                #print(item_list)

    #        with open("./review/{}.csv".format(series), encoding = 'cp932', errors = 'ignore') as fp:
    #            lst = []
    #            reader = csv.reader(fp)
    #            for row in reader:
    #                lst.append(row[2])#C列

            #print(lst)#listはレビューを係り受け解析し、csvにし、Ⅽ列（原形）を抜き出したもの

In [ ]:
for camera in cameras:
    for page in range(1,cameras[camera][1]+1): #range(1,4)は1,2,3なので＋1をする
        r = requests.get("https://review.kakaku.com/review/{0}/page={1}/#tab".format(cameras[camera][0],str(page))) #requestを使ってurlを呼び出す parserでrを分析  {0}{1}はformatの何番目にその要素があるか
        soup = BeautifulSoup(r.content, "html.parser") #BeautifulSoupのアルゴリズムを使う
        wordclass = soup.find_all("p","revEntryCont") #pでくくられたrevEntryCont（レビュー本文）を全部探す
        for person,item in enumerate(wordclass,1): #for i, name in enumerate(l, 1):  print(i, name)  1 Alice  2 Bob  3 Charlie　　
            file = open("./review/{0}_{1}_{2}.txt".format(camera,str(page),person), "r")
            review = file.read()
            #print(review)
            c = review.count("ボタン")
            if c > 1:
                okikae = str(c).replace( str(c) , '1') #strにすることで1などの数字は全部文字列型にする
                print(okikae)
            else:
                print(c)
            file.close()

In [ ]:
with open("./review/{}.csv".format(series), "w"):#csvファイルを作成
    pass#処理はしない

with open("./review/{}.csv".format(series), "a", newline="")as fp: #newline=""はcsvファイルの一行詰めるため "a"は追記可能
    writer = csv.writer(fp)

    for series in selected_products:#品名ごとに抜き出す
        files = glob.glob("./review/*/*/{}_*_*.txt".format(series)) #*に入るのは何でもいい
        for file in files:
            f = open(file,"r")
            review = f.read()
            response = post(review)#post(???)で係り受け解析をした結果を得ることができる
            response = ast.literal_eval(post(review))#文字列を辞書型に変換
            #print(response)
            f.close()

    #        with open("./review/{}_word.csv".format(series), "a", newline="")as fp: #newline=""はcsvファイルの一行詰めるため "a"は追記可能
    #            writer = csv.writer(fp)
    #            for element in response["result"]["chunks"]:
    #                tokens_item = element["tokens"]#品詞や原形を抜き出す（値のみ）
    #                writer.writerows(tokens_item)


            id_list = []
            head_list = []
            for element in response["result"]["chunks"]:
                #係り受け解析の結果から各要素を抜き出す
                id_item = element["id"]#係り先を抜き出す（値のみ）
                id_list.append(id_item)
                
                head_item = element["head"]#係り先を抜き出す（値のみ）
                head_list.append(head_item)
                
                tokens_item = element["tokens"]#品詞や原形を抜き出す（値のみ）

                #tokensの処理
                kihon = []
                for kihonkei in tokens_item:
                    if kihonkei[3] == "特殊":#品詞が特殊なら省く
                        continue
                    kihon.append(kihonkei[0])#言葉をリストに追加
                tokens = ''.join(kihon)#リストの中身を結合
                #print(tokens_item)        
                                
               
                word_list = [id_item, head_item, tokens]#配列型にする
                all_list = [word_list]#二次元配列にする
        ##        print(all_list)
                writer.writerows(all_list)

            for id_number in headlist
            #idhead = id_list+head_list
            print(head_list)
            
        
             #係り受け先の言葉と合体させる
#            if id_item == head_item:
#                print(tokens_item)
        
            
            
            
            
            
                    #print(element["head"])
                #print(item_list)

    #        with open("./review/{}.csv".format(series), encoding = 'cp932', errors = 'ignore') as fp:
    #            lst = []
    #            reader = csv.reader(fp)
    #            for row in reader:
    #                lst.append(row[2])#C列

            #print(lst)#listはレビューを係り受け解析し、csvにし、Ⅽ列（原形）を抜き出したもの